In [1]:
import pyarrow.parquet as pq
import pandas as pd
import os
import re
# remove warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import  parquet file as parquet
parquet = pq.read_table('/workspaces/AICoinXpert/algo/webscraping/results.parquet')
two_euros_parquet = pq.read_table('/workspaces/AICoinXpert/algo/webscraping/two_euros_df.parquet')
# Convert parquet to pandas dataframe
df = parquet.to_pandas()
df_two_euros = two_euros_parquet.to_pandas()

In [3]:
df_two_euros.describe()

,price,title,tirage,picture_url
count,100,100,100,100
unique,91,100,72,57
top,148,Monaco 2 Euro commémorative 2007 - 25e anniver...,Tirage: 1.500,No picture found
freq,2,1,6,44


In [4]:
# merge two euros dataframe with the main dataframe
df= pd.concat([df, df_two_euros], ignore_index=True)

In [5]:
df.head()

,price,title,tirage,picture_url
0,"1,74",Allemagne 1 Cent 2022 A,Tirage: - - - | seulement Brillant Universel d...,https://www.pieces-euro.tv/img02/thumb/Allemag...
1,"2,51",Allemagne 2 Cent 2022 A,Tirage: 42.200.000,https://www.pieces-euro.tv/img04/thumb/Allemag...
2,"2,27",Allemagne 5 Cent 2022 A,Tirage: 27.000.000,https://www.pieces-euro.tv/img05/thumb/Allemag...
3,"2,61",Allemagne 10 Cent 2022 A,Tirage: 19.800.000,https://www.pieces-euro.tv/img01/thumb/Allemag...
4,"2,24",Allemagne 20 Cent 2022 A,Tirage: 26.200.000,https://www.pieces-euro.tv/img01/thumb/Allemag...


In [6]:
# basic info
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3862 entries, 0 to 3861
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   price        3862 non-null   object
 1   title        3862 non-null   object
 2   tirage       3862 non-null   object
 3   picture_url  3862 non-null   object
dtypes: object(4)
memory usage: 120.8+ KB


In [7]:
df.count()

price          3862
title          3862
tirage         3862
picture_url    3862
dtype: int64

In [8]:
# check for null values and duplicates
df.isnull().sum(), print(f'duplicates : {df.duplicated().sum()}')

duplicates : 234


(price          0
 title          0
 tirage         0
 picture_url    0
 dtype: int64,
 None)

### Pre-treatment

In [9]:
df['price'] = df['price'].str.replace(',', '.')
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [10]:
df['title'] = df['title'].astype(str)
#df['tirage'] = df['tirage'].astype(float)
#df.set_index('picture_url', inplace=True)

## Indexing 

In [11]:
# Read names of image_folder and assign their name as the index of df

path = '/workspaces/AICoinXpert/algo/webscraping/data/image_folder'
# Read the file names in order they are organized 

# Read the file names from the folder without sorting
file_names = sorted(os.listdir(path))

# Create a DataFrame with the file names
data = pd.DataFrame({'file_names': file_names})

# Print the DataFrame
data


,file_names
0,Allemagne-1-Cent-2002-A-4030-155548458652804.jpg
1,Allemagne-1-Cent-2003-A-4130-155548636420650.jpg
2,Allemagne-1-Cent-2004-A-4230-146382005395403.jpg
3,Allemagne-1-Cent-2005-A-4330-146382019089283.jpg
4,Allemagne-1-Cent-2006-A-4430-146382049379369.jpg
...,...
3508,Vatican-50-Cent-2020-3270400-159807850444196.jpg
3509,Vatican-50-Cent-2021-3313350-162401151586155.jpg
3510,Vatican-50-Cent-2022-3359750-165673348677571.jpg
3511,Vatican-50-Cent-2023-3403350-168506824994730.jpg


In [12]:
df["country"] = df["title"].str.split().str[0] # extract country of origin
# df["amount"] = df["title"].str.split().str[1] # get the amount of the coin
df['currency'] = df['title'].str.extract(r'(Cent|Euro)') # get currency of the coin
# extract coins amount that are in range from 1 to 60
df['amount'] = df['title'].str.extract(r'(\d{1,2})')

# Extract years between date ranges
df['year'] = df['title'].str.extract(r'\b(\d{4})\b')

# Extract numeric values from the specified column using regex
df["tirage"] = df['tirage'].str.extract(r'(\d{1,3}(?:\.\d{3})*(?:\.\d*)?)', expand=False)
# remove title column
df.drop(columns=['title'], inplace=True)


In [13]:

# Extract the index from the URL based on the country name and remove everything before the last backslash
df['Index'] = df.apply(lambda row: row['picture_url'].rsplit('/', 1)[-1], axis=1)

# Set the index of the DataFrame
df.set_index('Index', inplace=True)

# drop picture_url column
df.drop(columns=['picture_url'], inplace=True)

In [14]:
# function to check folder names that match to the index of the dataframe

def check_index_names(df, path_index):
    # Create a new column to store the check result
    df['SameIndex'] = df.index.isin(path_index)
    return df


check_index_names(df, data['file_names'])

,price,tirage,country,currency,amount,year,SameIndex
Index,,,,,,,
Allemagne-1-Cent-2022-A-3348000-164938774098273.jpg,1.74,NaN,Allemagne,Cent,1,2022,True
Allemagne-2-Cent-2022-A-3348050-164938774641157.jpg,2.51,42.200.000,Allemagne,Cent,2,2022,True
Allemagne-5-Cent-2022-A-3348100-164938775264159.jpg,2.27,27.000.000,Allemagne,Cent,5,2022,True
Allemagne-10-Cent-2022-A-3348150-164938775750568.jpg,2.61,19.800.000,Allemagne,Cent,10,2022,True
Allemagne-20-Cent-2022-A-3348200-164938776176727.jpg,2.24,26.200.000,Allemagne,Cent,20,2022,True
...,...,...,...,...,...,...,...
Vatican-2-Euro-commemorative-2007-80e-anniversaire-de-Sa-Saintete-le-pape-Benoit-XVI-Blister-2830090-153033184271567.jpg,151.00,85.000,Vatican,Euro,2,2007,True
Pays-Bas-2-Euro-35-ans-du-programme-Erasmus-2022-BE-3364550-165908203984129.jpg,151.00,2.500,Pays-Bas,Euro,2,2022,True
No picture found,150.00,10.000,France,Euro,2,2019,False


In [15]:
# count how many false in SameIndex column

df['SameIndex'].value_counts()

# display the false values

SameIndex
True     3801
False      61
Name: count, dtype: int64

In [16]:
df[df['SameIndex'] == False]

,price,tirage,country,currency,amount,year,SameIndex
Index,,,,,,,
No picture found,NaN,NaN,Allemagne,Euro,2,2018,False
No picture found,NaN,NaN,Allemagne,Euro,2,2015,False
No picture found,NaN,NaN,Allemagne,Euro,2,2013,False
No picture found,NaN,NaN,Allemagne,Euro,2,2012,False
No picture found,NaN,NaN,Allemagne,Euro,2,2009,False
...,...,...,...,...,...,...,...
No picture found,163.0,10.000,Portugal,Euro,2,2016,False
No picture found,160.0,10.000,France,Euro,2,2014,False
No picture found,160.0,10.000,France,Euro,2,2015,False


In [17]:
# display year only 2023 
df[df['year'] == '2023']

,price,tirage,country,currency,amount,year,SameIndex
Index,,,,,,,
Croatie-Serie-Euro-Dubrovnik-2023-3388650-167870704483484.jpg,30.03,30.000,Croatie,Euro,20,2023,True
noimage.png,34.07,1.000,Croatie,Euro,20,2023,True
Allemagne-1-Cent-2023-A-3390100-168265018226889.jpg,2.25,28.800.000,Allemagne,Cent,1,2023,True
Allemagne-2-Cent-2023-A-3390150-168265018554688.jpg,1.84,18.000.000,Allemagne,Cent,2,2023,True
Allemagne-5-Cent-2023-A-3390200-168265019061568.jpg,1.67,9.000.000,Allemagne,Cent,5,2023,True
...,...,...,...,...,...,...,...
Slovaquie-2-Euro-100e-anniversaire-de-la-premiere-transfusion-sanguine-en-Slovaquie-2023-Coincard-3394100-167782655710972.jpg,24.96,5.000,Slovaquie,Euro,2,2023,True
Slovaquie-2-Euro-100e-anniversaire-de-la-premiere-transfusion-sanguine-en-Slovaquie-2023-BE-3394150-167782656249877.jpg,104.36,5.000,Slovaquie,Euro,2,2023,True
No picture found,535.00,25.000,Monaco,Euro,2,2023,False


In [18]:
# check how many nans in the dataframe

df.isnull().sum()


price         17
tirage       412
country        0
currency      57
amount        36
year          38
SameIndex      0
dtype: int64

In [19]:
# Make Index a column named folder_path and reset the index
df.reset_index(inplace=True)
df.rename(columns={'Index': 'folder_path'}, inplace=True)

In [20]:
df

,folder_path,price,tirage,country,currency,amount,year,SameIndex
0,Allemagne-1-Cent-2022-A-3348000-16493877409827...,1.74,NaN,Allemagne,Cent,1,2022,True
1,Allemagne-2-Cent-2022-A-3348050-16493877464115...,2.51,42.200.000,Allemagne,Cent,2,2022,True
2,Allemagne-5-Cent-2022-A-3348100-16493877526415...,2.27,27.000.000,Allemagne,Cent,5,2022,True
3,Allemagne-10-Cent-2022-A-3348150-1649387757505...,2.61,19.800.000,Allemagne,Cent,10,2022,True
4,Allemagne-20-Cent-2022-A-3348200-1649387761767...,2.24,26.200.000,Allemagne,Cent,20,2022,True
...,...,...,...,...,...,...,...,...
3857,Vatican-2-Euro-commemorative-2007-80e-annivers...,151.00,85.000,Vatican,Euro,2,2007,True
3858,Pays-Bas-2-Euro-35-ans-du-programme-Erasmus-20...,151.00,2.500,Pays-Bas,Euro,2,2022,True
3859,No picture found,150.00,10.000,France,Euro,2,2019,False
3860,Espagne-2-Euro-commemorative-2014-Parc-Gueell-...,148.00,12.000,Espagne,Euro,2,2014,True


In [21]:
# remove the rows that have false in SameIndex column "only 61 rows"
df = df[df['SameIndex']]

### Types modification

In [22]:
df.tirage = df.tirage.str.replace('.', '').astype(float)
df['amount'] = pd.to_numeric(df['amount'], errors='coerce').astype(pd.Int64Dtype())
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype(pd.Int64Dtype())


## Handling missing values


In [23]:
df.isnull().sum() 
# Count the number of missing values in each column

folder_path      0
price            0
tirage         398
country          0
currency        57
amount          36
year            38
SameIndex        0
dtype: int64

In [24]:
# display the rows that have null values in tirage column
df[df['tirage'].isnull()].head(5)


,folder_path,price,tirage,country,currency,amount,year,SameIndex
0,Allemagne-1-Cent-2022-A-3348000-16493877409827...,1.74,NaN,Allemagne,Cent,1,2022,True
6,Allemagne-1-Euro-2022-A-3348300-16493877725905...,8.15,NaN,Allemagne,Euro,1,2022,True
7,Allemagne-2-Euro-2022-A-3348350-16493877782806...,16.19,NaN,Allemagne,Euro,2,2022,True
234,Autriche-1-Cent-2022-3353450-165129007877935.jpg,0.66,NaN,Autriche,Cent,1,2022,True
235,Autriche-2-Cent-2022-3353500-165129008473282.jpg,0.66,NaN,Autriche,Cent,2,2022,True


In [25]:
# function to get the average tirage of coins values per country
def fill_tirage_with_average(df):
    df['tirage'] = pd.to_numeric(df['tirage'], errors='coerce').astype(float)  # Convert 'tirage' column to float and handle NaN values
    
    for country in df['country'].unique():
        mean_tirage = df[df['country'] == country]['tirage'].mean()
        df.loc[(df['country'] == country) & (df['tirage'].isnull()), 'tirage'] = mean_tirage
        
    return df

# use the function to fill the null values in tirage column
df = fill_tirage_with_average(df)


In [26]:
### Check on currency column / display the rows that have null values in currency column
df[df['currency'].isnull()]
# remove all the rows that have null values in currency column because they are the Kit coins
df = df[df['currency'].notna()]

In [27]:
# display all rows with nan values in amount column
df[df['year'].isnull()]
# remove all the rows that have null values year column because they are not coins
df = df[df['year'].notna()]

In [28]:
# remove SameIndex column
df.drop(columns=['SameIndex'], inplace=True)

In [29]:
# display year under 1900
df[df['year'] < 1950]
# remove all the rows that have year under 1950
df = df[df['year'] > 1950]

In [30]:
# find all the folderpath with nonimage.png and remove them
df = df[df['folder_path'].str.contains('noimage.png') == False]

# create a column name with the 4 first elements in folder_path separated by '-'
df['coin_name'] = df['folder_path'].apply(lambda x: '-'.join(x.split('-')[:4]))

In [31]:
# check double values
duplicate = df[df.duplicated(subset=['coin_name', 'year', 'country'], keep=False)].sort_values(by='coin_name')
# display the duplicate rows
duplicate

,folder_path,price,tirage,country,currency,amount,year,coin_name
3294,Allemagne-1-Cent-2023-A-3390100-16826501822688...,2.25,2.880000e+07,Allemagne,Cent,1,2023,Allemagne-1-Cent-2023
3528,Allemagne-1-Cent-2023-A-3390100-16826501822688...,2.25,2.880000e+07,Allemagne,Cent,1,2023,Allemagne-1-Cent-2023
3300,Allemagne-1-Euro-2023-A-3390400-16826502092701...,6.74,4.597929e+07,Allemagne,Euro,1,2023,Allemagne-1-Euro-2023
3534,Allemagne-1-Euro-2023-A-3390400-16826502092701...,6.74,4.597929e+07,Allemagne,Euro,1,2023,Allemagne-1-Euro-2023
3297,Allemagne-10-Cent-2023-A-3390250-1682650195894...,2.06,1.840000e+07,Allemagne,Cent,10,2023,Allemagne-10-Cent-2023
...,...,...,...,...,...,...,...,...
3476,Vatican-5-Cent-2023-3403200-168506822546622.jpg,4.23,4.500000e+04,Vatican,Cent,5,2023,Vatican-5-Cent-2023
3258,Vatican-50-Cent-2005-2800350-145431916990497.jpg,21.85,8.500000e+04,Vatican,Cent,50,2005,Vatican-50-Cent-2005
3266,Vatican-50-Cent-2005-Sede-Vacante-MMV-2800430-...,32.46,6.000000e+04,Vatican,Cent,50,2005,Vatican-50-Cent-2005
3479,Vatican-50-Cent-2023-3403350-168506824994730.jpg,5.17,4.500000e+04,Vatican,Cent,50,2023,Vatican-50-Cent-2023


In [32]:
# keep only one row of the duplicate rows

#keepindex = duplicate.drop_duplicates(subset=['coin_name', 'year', 'country'], keep='first').index

In [33]:
# display how many coin names are duplicated
duplicate['coin_name'].value_counts()

coin_name
Allemagne-2-Euro-2023    22
France-2-Euro-Jeux       12
Slovaquie-2-Euro-100e     6
Italie-2-Euro-150e        6
Pays-Bas-2-Euro           6
                         ..
Grece-10-Cent-2002        2
Grece-10-Cent-2022        2
Grece-2-Cent-2002         2
Grece-2-Cent-2022         2
Vatican-50-Cent-2023      2
Name: count, Length: 235, dtype: int64

In [34]:
# Keeping only one example of each coin name and remove the rest
df.drop_duplicates(subset=['coin_name', 'year', 'country'], keep='first', inplace=True)

In [35]:
df[df.duplicated(subset=['coin_name', 'year', 'country'], keep=False)].sort_values(by='coin_name')

,folder_path,price,tirage,country,currency,amount,year,coin_name


In [36]:
# save the dataframe to csv file as 'cleaned_data.csv'
df.to_csv('/workspaces/AICoinXpert/algo/webscraping/cleaned_data_casual_list.csv', index=False)